In [1]:
from utils_nlp import Tools
tools = Tools('mhk9c')

In [2]:
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline
import sparknlp
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.clustering import LDA

from pyspark.sql.functions import col,lit

In [3]:
import sparknlp
spark = sparknlp.start()

In [23]:
_df = tools.load_data(spark, "russian-troll-tweets-enriched")
df = _df.filter(_df['label']==0)
df = df.withColumn("text", col("content"))

Done loading from /project/ds5559/team1_sp22/data//russian-troll-tweets-enriched.


In [24]:
df.select('text').show(5,False)

+---------------------------------------------------------------------------------------------------------------+
|text                                                                                                           |
+---------------------------------------------------------------------------------------------------------------+
|RT GoogleStreetArt: Street Art by Weslart in La Bañeza Spain https://t.co/LtHrdvIBCH                           |
|New Street Art by Osgemeos https://t.co/AbHowZ6gsR                                                             |
|Street Art by Weslart in La Bañeza Spain https://t.co/LtHrdvIBCH                                               |
|RT GoogleStreetArt: Sean Yoro using tide levels to reveal unique Street Art twice a day https://t.co/er5ySndnkf|
|Newest Street Art piece in Salem mass for UrbanArtMuseum by chorboogie https://t.co/QQEPv25zdO                 |
+---------------------------------------------------------------------------------------

In [25]:
document_assembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

use = UniversalSentenceEncoder.pretrained('tfhub_use', lang="en") \
  .setInputCols(["document"])\
  .setOutputCol("sentence_embeddings")

document_classifier = ClassifierDLModel.pretrained('classifierdl_use_fakenews', 'en') \
  .setInputCols(["document", "sentence_embeddings"]) \
  .setOutputCol("class")

nlpPipeline = Pipeline(stages=[document_assembler, use, document_classifier])

# light_pipeline = LightPipeline(nlpPipeline.fit(spark.createDataFrame([['']]).toDF("text")))

# annotations = light_pipeline.fullAnnotate('Donald Trump a KGB Spy? 11/02/2016 In today’s video, Christopher Greene of AMTV reports Hillary Clinton')

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
classifierdl_use_fakenews download started this may take some time.
Approximate size to download 21.4 MB
[OK!]


In [17]:
type(nlpPipeline)

pyspark.ml.pipeline.Pipeline

In [26]:
model = nlpPipeline.fit(df)

In [27]:
prediction = model.transform(df)

In [29]:
prediction.select('text','class').show(5, False)

+---------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------+
|text                                                                                                           |class                                                                                   |
+---------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------+
|RT GoogleStreetArt: Street Art by Weslart in La Bañeza Spain https://t.co/LtHrdvIBCH                           |[{category, 0, 83, FAKE, {sentence -> 0, FAKE -> 1.0, REAL -> 1.7270991E-9}, []}]       |
|New Street Art by Osgemeos https://t.co/AbHowZ6gsR                                                             |[{category, 0, 49, FAKE, {sentence -> 0, FAKE -> 1.0, REAL -> 9.623411E-14}

In [31]:
light_pipeline = LightPipeline(nlpPipeline.fit(spark.createDataFrame([['']]).toDF("text")))

annotations = light_pipeline.fullAnnotate('Donald Trump a KGB Spy? 11/02/2016 In today’s video, Christopher Greene of AMTV reports Hillary Clinton')
annotations.select

[{'document': [Annotation(document, 0, 102, Donald Trump a KGB Spy? 11/02/2016 In today’s video, Christopher Greene of AMTV reports Hillary Clinton, {})],
  'sentence_embeddings': [Annotation(sentence_embeddings, 0, 102, Donald Trump a KGB Spy? 11/02/2016 In today’s video, Christopher Greene of AMTV reports Hillary Clinton, {'sentence': '0', 'token': 'Donald Trump a KGB Spy? 11/02/2016 In today’s video, Christopher Greene of AMTV reports Hillary Clinton', 'pieceId': '-1', 'isWordStart': 'true'})],
  'class': [Annotation(category, 0, 102, FAKE, {'sentence': '0', 'FAKE': '0.99999917', 'REAL': '8.4841463E-7'})]}]